In [ ]:
import sys
from pathlib import Path

# assume this notebook lives in regiem_45_model/eval_45/
PROJECT_ROOT = Path().resolve().parent.parent  # two levels up
sys.path.insert(0, str(PROJECT_ROOT))

# now Python will find regiem_45_model as a top-level package
from regiem_45_model.pre_proc_labelling_long import plot_all_regimes_long

import pandas as pd
from pathlib import Path
import numpy as np

'''
python -m regiem_45_model.eval_45.compare
'''

df = pd.read_csv("predictions.csv", index_col=0)

inferenced = df.to_numpy()
print(type(inferenced), inferenced.shape)

#Need to slice off front 159 from this.

labels_list = []
for i in range(50):
    print(f"Processing graph #{i+1}/50")           
    one_d = plot_all_regimes_long(750, False, i)
    labels_list.append(one_d)

labels2d = np.vstack(labels_list)
print("Final shape:", labels2d.shape) 


inferenced_cut = inferenced[:, :-9]

inferenced_cut = np.where(inferenced_cut > 0.5, 2, 0) #If val > 0.5 then Bear, else Bull
auto_labels_cut     = labels2d[:, 159:]

print(f"inferenced: {inferenced.shape} → {inferenced_cut.shape}")
print(f"labels2d:   {labels2d.shape}   → {auto_labels_cut.shape}")





